In [1]:
!pip install pyspark

import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Uygulama_1').getOrCreate()

spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=f1d5eed94ef7f656ac8b037a4da586466a7fbe8349402350275054fd02e1d019
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
df=spark.read.csv("dataMissed.csv",header=True,inferSchema=True)
df.show()

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
| Adem|  37|        10| 10000|
| Banu|  24|         8| 11000|
| Esat|  46|         5|  NULL|
|Erdal|  47|         4|  8900|
|Zehra|  33|         9| 14200|
| Naim|NULL|      NULL|  5000|
| NULL|  23|         2|  4000|
| NULL|  41|        11|  NULL|
+-----+----+----------+------+



In [3]:
df.na.drop().show() # null gördüğü tüm satırları atıyor

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
| Adem| 37|        10| 10000|
| Banu| 24|         8| 11000|
|Erdal| 47|         4|  8900|
|Zehra| 33|         9| 14200|
+-----+---+----------+------+



In [4]:
# en az 3 sütun dolu ise tut aksi takdirde at

df.na.drop(how="any",thresh=3).show() # how=any -hangi sütun olursa olsun
                                      # dolu olma eşiği

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
| Adem| 37|        10| 10000|
| Banu| 24|         8| 11000|
| Esat| 46|         5|  NULL|
|Erdal| 47|         4|  8900|
|Zehra| 33|         9| 14200|
| NULL| 23|         2|  4000|
+-----+---+----------+------+



In [6]:
df.na.drop(subset="name").show() # sadece name sütununda null olanları atmasını istiyoruz

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
| Adem|  37|        10| 10000|
| Banu|  24|         8| 11000|
| Esat|  46|         5|  NULL|
|Erdal|  47|         4|  8900|
|Zehra|  33|         9| 14200|
| Naim|NULL|      NULL|  5000|
+-----+----+----------+------+



In [7]:
df.na.drop(subset=["name","experience"]).show() # köşeli parantezi gruplama için kullandık.

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
| Adem| 37|        10| 10000|
| Banu| 24|         8| 11000|
| Esat| 46|         5|  NULL|
|Erdal| 47|         4|  8900|
|Zehra| 33|         9| 14200|
+-----+---+----------+------+



In [15]:
from pyspark.ml.feature import Imputer # null değerleri tamamlamak için feature özelliğini import edip ımputer objesini kullanacağız
imputer=Imputer(inputCol="Salary",outputCol="Salary").setStrategy("mean") # ortalama şeklinde doldurmasını istedik

In [16]:
df2 = imputer.fit(df).transform(df) # fit datayı nerden alacağını gösterir. transform ise hangi dataya uygulayacağımızı söyler.

In [17]:
df2.show()

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
| Adem|  37|        10| 10000|
| Banu|  24|         8| 11000|
| Esat|  46|         5|  8850|
|Erdal|  47|         4|  8900|
|Zehra|  33|         9| 14200|
| Naim|NULL|      NULL|  5000|
| NULL|  23|         2|  4000|
| NULL|  41|        11|  8850|
+-----+----+----------+------+



In [18]:
#aggregation operators

df.agg({"Salary":"mean"}).show()

+-----------+
|avg(Salary)|
+-----------+
|     8850.0|
+-----------+



In [19]:
df.agg({"Salary":"stddev"}).show()

+-----------------+
|   stddev(Salary)|
+-----------------+
|3818.769435302425|
+-----------------+

